In [5]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import skew


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

train["SalePrice"] = np.log1p(train["SalePrice"])

numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.69]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

all_data = pd.get_dummies(all_data)

all_data = all_data.fillna(all_data.mean())

X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice


#### models select

from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 7.8)



from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.0005)

model = lasso

### predict
model.fit(X_train, y)

preds = np.expm1(model.predict(X_test))
solution = pd.DataFrame({"id":test.Id, "SalePrice":preds})
solution.to_csv("ridge14.csv", index = False)